In [1]:
import keras
from matplotlib import pyplot as plt 
import numpy as np
import pandas as pd 
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization,MaxPool2D
from keras import Sequential
import np_utils
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

## Step 1 - Building the CNN

#### Initializing the CNN


In [2]:
model= Sequential()

### Creating a convolutional network and adding pooling


In [3]:
# Creating a Sequential model
#1st layer and pooling
model.add(Conv2D(75 , (3,3) , activation = 'relu' , input_shape = (64,64,1)))
#model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
#model.add(Dropout(0.3))

model.add(Conv2D(50 , (3,3) ,  activation = 'relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
#model.add(Dropout(0.3))

model.add(Conv2D(25 , (3,3) , activation = 'relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
#model.add(Dropout(0.3))



### Adding final layer 

In [4]:
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))

opt = keras.optimizers.Adam(learning_rate=0.001)
model.add(Dense(units = 4 , activation = 'softmax'))


#### Compiling the model

In [5]:
model.compile(optimizer = opt , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 75)        750       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 75)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 50)        33800     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 50)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 25)        11275     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 25)          0         
_________________________________________________________________
flatten (Flatten)            (None, 900)               0

## Step 2 - Preparing the train/test data and training the model

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen =ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True)
test_datagen =ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(64, 64),
        batch_size=20,
        color_mode='grayscale',
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(64, 64),
        batch_size=20,
        color_mode='grayscale',
        class_mode='categorical')


Found 15603 images belonging to 4 classes.
Found 2035 images belonging to 4 classes.


### training the model

In [7]:
model.fit(
        train_generator,
        steps_per_epoch=9947//20,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=2035//20
        )

Epoch 1/10
497/497 [==============================] - 60s 119ms/step - loss: 0.2274 - accuracy: 0.9097 - val_loss: 1.0718 - val_accuracy: 0.7272
Epoch 2/10
497/497 [==============================] - 53s 106ms/step - loss: 0.0014 - accuracy: 0.9995 - val_loss: 0.3553 - val_accuracy: 0.8550
Epoch 3/10
497/497 [==============================] - 53s 106ms/step - loss: 0.0070 - accuracy: 0.9979 - val_loss: 0.5609 - val_accuracy: 0.8347
Epoch 4/10
497/497 [==============================] - 52s 104ms/step - loss: 0.0050 - accuracy: 0.9984 - val_loss: 0.4419 - val_accuracy: 0.8876
Epoch 5/10
497/497 [==============================] - 52s 105ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.7946 - val_accuracy: 0.8178
Epoch 6/10
497/497 [==============================] - 50s 100ms/step - loss: 1.1725e-04 - accuracy: 1.0000 - val_loss: 0.6267 - val_accuracy: 0.8446
Epoch 7/10
497/497 [==============================] - 49s 99ms/step - loss: 5.6263e-05 - accuracy: 1.0000 - val_loss: 0.5406 -

#### Saving the model

In [8]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('model-bw.h5')